In [1]:
# General
import pandas as pd
import numpy as np
import polars as pl
# from pandarallel import pandarallel
# pandarallel.initialize()

from datetime import datetime, timedelta, date
import json
import ast
import glob
import os
import pickle
import re
from itertools import groupby
import time

# Visualization
import matplotlib.pyplot as plt
import seaborn as sns

# # Bigquery
# from google.oauth2 import service_account
# from google.cloud import bigquery
# from google.cloud import bigquery_storage

import warnings
warnings.filterwarnings('ignore')

import pyarrow.parquet as pq
import pyarrow.fs as fs
import dask.dataframe as dd
import vaex

pd.options.display.float_format = '{:,.2f}'.format
pd.set_option('display.max_columns', None)
pd.set_option('display.max_columns', 500)
pd.set_option('display.max_colwidth', -1)  # or 199
# pd.set_option("display.max_rows", None) 

In [2]:
# define function here
def f(x):
    try:
        return ast.literal_eval(str(x))   
    except:
        return []

In [3]:
parquet_files = ['Data_stroke.parquet']
df = vaex.open(parquet_files).to_pandas_df()
df.head()

visits    rowID  \
0  2       1000135   
1  1       1000246   
2  2       1000274   
3  2       1000311   
4  1       100039    

                                                                cid class  \
0  b472531d4caaecc893dc345f31ee8317ea459926dd37f0da1c50d2af2e8543aa  AMB    
1  1cf272b413de5fc4e1166e3b5aa35e146125efd0f33ad13a73f58368da638167  AMB    
2  239378b8bc40473f3f73235f6306c0ebb1d50af8aae434de8f75f0bac4fcd742  AMB    
3  b360f3bf1549170ef3d569c7f618f004bd2b78d210b8c7f80000ba75718cf01c  AMB    
4  4bad23f241ec47aeb6f333bbc87a7b4da1b6ded6969d21789ddb638be41ea985  AMB    

  visitDateTime hospCode             diagText  \
0 2022-08-16     10832    cerebral infarction   
1 2020-03-02     10685    None                  
2 2020-03-02     10753    None                  
3 2022-08-01     10832    old CVA, HT           
4 2021-01-14     10697    None                  

                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                       diagnoses  \
0  [{"icd10":"I639","icd10Name":"cerebral infarction;unspecified","diagType":"1","diagTypeName":""}]                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                               
1  [{"icd10":"I610","icd10Name":"Intracerebral haemorrhage in hemisphere, subcortical","diagType":"1","diagTypeName":"Principal Diagnosis"},{"icd10":"I10","icd10Name":"Essential (primary) hypertension","diagType":"2","diagTypeName":"Comorbidity (โรคอื่นที่เป็นร่วมด้วย)"},{"icd10":"E785","icd10Name":"Hyperlipidaemia, unspecified","diagType":"2","diagTypeName":"Comorbidity (โรคอื่นที่เป็นร่วมด้วย)"},{"icd10":"Z133","icd10Name":"Special screening examination for mental and behavioural disorders","diagType":"4","diagTypeName":"Other (โรคอื่น ๆ)"}]                                                                                                                              
2  [{"icd10":"I64","icd10Name":"Stroke,not specified as haemorrhage or infarction","diagType":"1","diagTypeName":"Principal Diagnosis"},{"icd10":"E149","icd10Name":"U-DM Without complications","diagType":"2","diagTypeName":"Comorbidity (โรคอื่นที่เป็นร่วมด้วย)"},{"icd10":"I10","icd10Name":"Essential (primary) hypertension","diagType":"2","diagTypeName":"Comorbidity (โรคอื่นที่เป็นร่วมด้วย)"},{"icd10":"I251","icd10Name":"Atherosclerotic heart disease","diagType":"2","diagTypeName":"Comorbidity (โรคอื่นที่เป็นร่วมด้วย)"},{"icd10":"E789","icd10Name":"Disorder of lipoprotein metabolism,unspecified","diagType":"2","diagTypeName":"Comorbidity (โรคอื่นที่เป็นร่วมด้วย)"}]   
3  [{"icd10":"I64","icd10Name":"CVA","diagType":"1","diagTypeName":""},{"icd10":"I10","icd10Name":"blood pressure high (see also hypertension)     ***** ยกเว้นผู้ป่วยตั้งครรภ์ คลอด หลังคลอด ******","diagType":"2","diagTypeName":""},{"icd10":"R42","icd10Name":"Dizziness","diagType":"2","diagTypeName":""}]                                                                                                       

In [4]:
df.shape

(111011, 76)

In [5]:
_col = ['visits','rowID','cid','vital_cc','vital_pe', 'visitDateTime',"icd10_list",'diagType_1','diagType_2','diagType_3','patientAge', "patientSexName", "smoking", "narcotic", "exercise",
 "alcohol","vital_bw","vital_waist", "vital_height","vital_bmi",'vital_hr','vital_pulse','vital_temperature','vital_rr',"vital_bps", "vital_bpd"]
data_col = df.loc[:, _col]

In [6]:
# optimized version of extract_labResults fucntion
# Using the .get() method to access dictionary values with a default value, instead of using indexing. This way, the code doesn't raise an error when the key is missing.
# Using the continue statement to skip to the next iteration in the for loops when the conditions are not met. This avoids having to indent the rest of the code inside the loops.

def extract_labResults(lab_list_dict):
    result = {} 
    for i, dictionary in enumerate(lab_list_dict): # enumerate เพิ่มตัวเลขจำนวนการลูป
        lab_head_data = dictionary.get('labHeadData', {}) # ไปเอา value ของ key labHeadData มา ถ้าไม่มีค่าก็เป็น {}
        if lab_head_data.get('confirmReport') != 'Y':
            continue # ถ้าไม่ Y ก็ไม่ต้องทำต่อ (ข้ามไปเลย) ไปทำใหม่
        result[i] = {
            'departmentName': lab_head_data.get('departmentName'), 
            'spcltyName': lab_head_data.get('spcltyName'),
            'labReportData': {}
        }
        lab_list = dictionary.get('labReportData', []) # ไปเอา value ของ key labReportData มา ถ้าไม่มีค่าก็เป็น {}
        for d in lab_list:
            if d.get('confirm') != 'Y':
                continue
            result[i]['labReportData'][d['labItemsCode']] = {
                'labItemsNameRef': d['labItemsNameRef'],
                'labOrderResult': d['labOrderResult'],
                'labItemsNormalValueRef': d['labItemsNormalValueRef'],
                'labItemsUnit': d['labItemsUnit']
            }
    return result

In [7]:
def clean_labResults(lab_result_2):
    '''
    To clean these 3 -> labOrderResult, labItemsNormalValueRef, labItemsUnit
    
    [labOrderResult] # เอาเเค่ตัวเลข (ค่าเเลป) ไม่เอาช่วง
    Mostly labOrderResult is filled with number or string pattern but some of it has like "0 - 0 Unit" pattern
    which it may be the labItemsNormalValueRef and labItemsUnit(Obviously not supposed to be in labOrderResult).
    So, in "0 - 0 Unit" case, we can move it to both of them if it is empthy or "-".
    
    Moreover, In labOrderResult, we'll only keep the number value (e.g.63.19 (Stage 2) -> use 63.19).
    
    [labItemsNormalValueRef] # ไม่เอา unit 
    "0 - 0" , "0 - 0 Unit" and string should be a pattern of labItemsNormalValueRef 
    but sometimes, labItemsNormalValueRef is exactly like labItemsUnit.
    So, if we can't extract it by using the regex labItemsNormalValueRef pattern and they're exacly alike.
    then we'll remove the labItemsNormalValueRef and leave the labItemsUnit there.
    '''
    
    number_unit_regex = r'^(-?\d+(?:[\.\,]\d+)?\s*-\s*-?\d+(?:[\.\,]\d+)?|[>=<]*\s*-?\d+(?:[\.\,]\d+)?)(.*)'

    for labHeadDataKey, labHeadData in lab_result_2.items():
        for labItemsCode, lab_data in labHeadData['labReportData'].items():
            # Extract number and unit from labOrderResult
            lab_order_result = lab_data['labOrderResult'].strip()
            lab_order_result_match = re.match(number_unit_regex, lab_order_result)
            if lab_order_result_match:
                try:
                    lab_data['labOrderResult'] = lab_order_result_match.group(1).strip()
                    if "stage" not in lab_order_result_match.group(2).lower():
                        lab_data['labItemsUnit'] = lab_order_result_match.group(2).strip()
                except IndexError:
                    pass

            # Extract number and unit from labItemsNormalValueRef
            lab_items_normal_value_ref = lab_data['labItemsNormalValueRef'].strip()
            lab_items_normal_value_ref_match = re.match(number_unit_regex, lab_items_normal_value_ref)
            if lab_items_normal_value_ref_match:
                try:
                    lab_data['labItemsNormalValueRef'] = lab_items_normal_value_ref_match.group(1)
                    if (lab_data['labItemsUnit'] == '' or lab_data['labItemsUnit'] == '-') and "stage" not in lab_items_normal_value_ref_match.group(2).lower():
                        lab_data['labItemsUnit'] = lab_items_normal_value_ref_match.group(2)
                except IndexError:
                    pass
            elif lab_data['labItemsNormalValueRef'].strip().lower() == lab_data['labItemsUnit'].strip().lower():
                lab_data['labItemsNormalValueRef'] = ''

    return lab_result_2


In [8]:
def extract_labResults_data(data):
    df = data.copy()
    df['lab_result_2'] = df['labResults'].apply(json.loads).apply(extract_labResults)
    return df

def clean_lab_result_data(data):
    df = data.copy()
    df['lab_result_3'] = df['lab_result_2'].apply(clean_labResults)
    return df

def extract_labResults3(df):
    lab_result = df.get("labResults")
    if lab_result and lab_result.get(df["labHeadData"]):
        return lab_result[df["labHeadData"]]
    return None

def extract_labReportData(df):
    if pd.isna(df['labReportData']) != True:
        for i in range(len(list(df['labReportData'].items()))):
            if list(df['labReportData'].items())[i][0] == df['labItemCode']:
                return list(df['labReportData'].items())[i][1]
    else:
        return df['labReportData']
    
def clean(df):
    df2 = extract_labResults_data(df)
    df2 = clean_lab_result_data(df2)
    df2.drop(['labResults','lab_result_2'], axis=1, inplace=True)
    df2.rename(columns = {'lab_result_3' : 'labResults'},inplace=True)
    return df2

def clean2(df2):
    ex = df2.explode('labResults')
    ex.rename(columns = {'labResults' : 'labHeadData'},inplace=True)
    df_merge = ex.merge(df2, how = 'left' , on=['visits','rowID','cid','visitDateTime'])
    df_merge['labHeadData2'] = df_merge.apply(extract_labResults3, axis=1)
    df_merge['labHeadData2'].fillna(value = np.NaN, inplace=True)
    df_merge['labHeadData2'] = df_merge['labHeadData2'].apply(lambda x: {} if pd.isna(x) else x)
    df_concat = pd.concat([df_merge[['visits','rowID', 'cid', 'visitDateTime','labHeadData']], pd.json_normalize(df_merge['labHeadData2'], max_level = 0)], axis=1)
    df_concat_ex = df_concat.explode('labReportData').reset_index(drop=True)
    df_concat_ex.rename(columns = {'labReportData':'labItemCode'}, inplace=True)
    df_merge2 = df_concat_ex.merge(df_concat, how = 'left' , on=['visits','rowID','cid','visitDateTime','labHeadData','departmentName','spcltyName'])
    df_merge2['labReportData2'] = df_merge2.apply(extract_labReportData, axis=1)
    df_merge2.drop(['labReportData'], axis=1,inplace=True)
    df_merge2.rename(columns={'labReportData2':'labReportData'}, inplace=True)
    df_concat2 = pd.concat([df_merge2[['visits','rowID','cid','visitDateTime','labHeadData','departmentName','spcltyName','labItemCode']], pd.json_normalize(df_merge2['labReportData'])], axis=1)
    df_concat2 = df_concat2.reindex(columns = ['visits','rowID','cid','visitDateTime','labHeadData','departmentName',
       'spcltyName','labItemCode','labItemsNameRef','labOrderResult','labItemsNormalValueRef','labItemsUnit'])
    return df_concat2

In [9]:
%%time
df2 = clean(df)
df_concat2 = clean2(df2)
df_concat2

Wall time: 1min 15s


,visits,rowID,cid,visitDateTime,labHeadData,departmentName,spcltyName,labItemCode,labItemsNameRef,labOrderResult,labItemsNormalValueRef,labItemsUnit
0,2,1000135,b472531d4caaecc893dc345f31ee8317ea459926dd37f0da1c50d2af2e8543aa,2022-08-16,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,1,1000246,1cf272b413de5fc4e1166e3b5aa35e146125efd0f33ad13a73f58368da638167,2020-03-02,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,2,1000274,239378b8bc40473f3f73235f6306c0ebb1d50af8aae434de8f75f0bac4fcd742,2020-03-02,0,งานหน้าห้องตรวจโรคทั่วไป 1,อายุรกรรม GP,1001,GLU,231,74.00 - 106.00,
3,2,1000311,b360f3bf1549170ef3d569c7f618f004bd2b78d210b8c7f80000ba75718cf01c,2022-08-01,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,1,100039,4bad23f241ec47aeb6f333bbc87a7b4da1b6ded6969d21789ddb638be41ea985,2021-01-14,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...
651096,1,999629,9638a995f2836643b505771c00668f0d6951b1084f10b5735051dd85ffc51534,2020-03-02,0,ฝ่ายเวชระเบียน,ชลอไต,5287,Tear drop,,,
651097,1,999629,9638a995f2836643b505771c00668f0d6951b1084f10b5735051dd85ffc51534,2020-03-02,0,ฝ่ายเวชระเบียน,ชลอไต,5288,Stomatocyte,,,
651098,1,999629,9638a995f2836643b505771c00668f0d6951b1084f10b5735051dd85ffc51534,2020-03-02,0,ฝ่ายเวชระเบียน,ชลอไต,5334,eGFR,48.02,,
651099,5,999834,507b2eed38b70f8f1f122b11c869b60713c95a4e4dbfaddf572bcbe18f00320e,2022-08-17,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [10]:
#before
df_concat2['labItemsNameRef'].value_counts()[0:50]

Creatinine             17524
BUN                    12639
MCH                    10940
MCV                    10940
MCHC                   10596
RBC                    9725 
Monocyte               9359 
Lymphocyte             9359 
Triglyceride           9232 
Potassium              9044 
Eosinophil             9015 
Basophil               9015 
eGFR                   8659 
Sodium                 8224 
Fasting glucose        8181 
Chloride               8007 
e-GFR (CKD-EPI)        7867 
e-GFR (MDRD)           7867 
Cholesterol            7781 
Neutrophil             7394 
WBC                    7304 
INR                    7080 
LDL-chelesterol        5946 
HGB                    5766 
PT                     5705 
HCT                    5396 
Bicarbonate            4697 
HbA1c                  4558 
SGPT (ALT)             4223 
absolute neutrophil    4084 
SGOT (AST)             4064 
Platelet (count)       4023 
Platelet (smear)       4023 
Hb                     3470 
RDW-CV        

In [11]:
# replace_values = open("replace_Value.txt", "r")
# df_concat2.replace({"labItemsNameRef": replace_values}, inplace=True)

In [12]:
%%time
# replace name lab 
replace_values = json.loads(open('replace_values_lab.json', 'r', encoding="utf8").read()) # utf8 : basic ตัวอักษร
df_concat2.replace({'labItemsNameRef': replace_values}, inplace=True)

Wall time: 5min 34s


In [13]:
# after
df_concat2['labItemsNameRef'].value_counts()[:22] # look lab top 50 

Creatinine         22544
FBS                21228
RBC                16231
WBC                16074
LDL-Cholesterol    14580
Potassium          14578
BUN                13470
HCT                13172
CO2                13147
Sodium             12513
PLT                12279
MCV                11948
MCH                11947
Triglyceride       11752
MCHC               11497
Basophil           10938
Neutrophil         10928
Lymphocytes        10920
Monocytes          10920
Eosinophil         10920
Chloride           10315
Cholesterol        9883 
Name: labItemsNameRef, dtype: int64

## Check labItemsUnit 

In [25]:
selected_lab_df = df_concat2.loc[:, ["rowID", "cid", "labItemsNameRef","labItemsNormalValueRef", "labOrderResult","labItemsUnit"]]

In [50]:
unique_labItemsNameRef = selected_lab_df.groupby(['labItemsNameRef','labItemsNormalValueRef','labItemsUnit']).agg(['nunique']).reset_index(drop=False)
unique_labItemsNameRef.loc[unique_labItemsNameRef['labItemsNameRef']=="HCT"]

,labItemsNameRef,labItemsNormalValueRef,labItemsUnit,rowID,cid,labOrderResult
,,,,nunique,nunique,nunique
1267,HCT,,,6,6,6
1268,HCT,,test,1,1,1
1269,HCT,%,,1,1,1
1270,HCT,20.0-70.0,%,7,7,5
1271,HCT,24 - 61,,87,68,74
...,...,...,...,...,...,...
1326,HCT,"M40-54,F37-47",,6,6,6
1327,HCT,"M:(41-51),F:(37-47)",,1,1,1
1328,HCT,"M:40-50, F:36-45 %",,2,2,2


In [39]:
unique_labItemsNameRef.iloc[:, 0:4]

,labItemsNameRef,labItemsNormalValueRef,labItemsUnit,rowID
,,,,nunique
0,,,,42
1,% A,%,-,1
2,% A2,2.0 - 3.5,%,1
3,% E,%,-,1
4,% F,<1.0,%,1
...,...,...,...,...
3728,ครั้งที่ 1,,-,1
3729,ตำแหน่งรอยโรค,,,1
3730,วิธีที่ 1,,COI,12


In [51]:
unique_labItemsNameRef.loc[unique_labItemsNameRef['labItemsNameRef']=="HCT"].value_counts()

(labItemsNameRef, )  (labItemsNormalValueRef, )  (labItemsUnit, )  (rowID, nunique)  (cid, nunique)  (labOrderResult, nunique)
HCT                                                                6                 6               6                            1
                                                 test              1                 1               1                            1
                     37.0 - 52.0                  %                2                 2               2                            1
                     37.0 - 54.0                                   1323              800             290                          1
                                                 %                 2                 2               2                            1
                                                                                                                                 ..
                     36 - 48                      %                179           

In [52]:
selected_lab_df.loc[selected_lab_df['labItemsNameRef'] == 'HCT'].value_counts()

rowID    cid                                                               labItemsNameRef  labItemsNormalValueRef  labOrderResult  labItemsUnit
412      f06f75bd580593ca11314220e00525d723da03d800cf239999533a2fd1b467b4  HCT              40 - 54                 25               %              5
648      09497d5a621951d3466eac152fdc1d0c710d16f2c8df71a78505339a542dae55  HCT              40 - 54                 24               %              4
479      829d5efd1e3b2fbdd145136232ffded6627063f30e15821c1e72c8958de008c5  HCT              40 - 54                 24               %              4
648      09497d5a621951d3466eac152fdc1d0c710d16f2c8df71a78505339a542dae55  HCT              40 - 54                 27               %              4
412      f06f75bd580593ca11314220e00525d723da03d800cf239999533a2fd1b467b4  HCT              40 - 54                 26               %              4
                                                                                                         

In [58]:
selected_lab_df.loc[(selected_lab_df['labItemsNameRef'] == 'Potassium'), 'labItemsUnit'].value_counts()

 mmol/L     8677
            3572
 mmol/l     2282
mmol/L      23  
mmol/l      17  
(R)         3   
Repeated    2   
R           1   
repeate     1   
Name: labItemsUnit, dtype: int64

In [18]:
asas

NameError: name 'asas' is not defined

In [ ]:
# change labItemsUnit to the top value_count for the Top 50 lab
for i in df_concat2['labItemsNameRef'].value_counts()[0:50].index:
    df_concat2['labItemsUnit'].loc[df_concat2['labItemsNameRef'] == i] = df_concat2[
        'labItemsUnit'].loc[df_concat2['labItemsNameRef'] == i].value_counts().keys()[0]
    
# change labItemsNormalValueRef to the top value_count for the Top 50 lab
for i in df_concat2['labItemsNameRef'].value_counts()[0:50].index:
    df_concat2['labItemsNormalValueRef'].loc[df_concat2['labItemsNameRef'] == i] = df_concat2[
        'labItemsNormalValueRef'].loc[df_concat2['labItemsNameRef'] == i].value_counts().keys()[0]

In [ ]:
# checking
df_concat2[df_concat2['labItemsNameRef'] == 'Creatinine']

In [ ]:
df_concat2

In [ ]:
df_concat2.dropna(subset = ['labHeadData'])

In [ ]:
top_lab = df_concat2['labItemsNameRef'].value_counts()[:22] # look lab top 22
top_lab

In [ ]:
# _selected_lab = ['Creatinine', 'FBS' , 'RBC', 'WBC' ,'LDL-Cholesterol','Potassium','BUN',                 
# 'HCT', 'CO2', 'Sodium' ,'PLT', 'MCV', 'MCH', 'Triglyceride', 'MCHC' ,'Basophil' ,'Neutrophil', 'Eosinophil',
# 'Monocytes' ,'Lymphocytes' ,'Chloride', 'Cholesterol' ,'eGFR' ,'PT', 'HDL-Cholesterol', 'eGFR (CKD-EPI)',
# 'eGFR (MDRD)', 'INR', 'HbA1c', 'HGB', 'SGPT', 'SGOT', 'Absolute', 'Neutrophil', 'Albumin', 'DTX'
#                 ]
# data = df_concat2.loc[df_concat2["labItemsNameRef"].isin(_selected_lab)]

In [ ]:
_selected_lab = ['Creatinine', 'FBS' , 'RBC', 'WBC' ,'LDL-Cholesterol','Potassium','BUN',                 
'HCT', 'CO2', 'Sodium' ,'PLT', 'MCV', 'MCH', 'Triglyceride', 'MCHC' ,'Basophil' ,'Neutrophil', 'Eosinophil',
'Monocytes' ,'Lymphocytes' ,'Chloride', 'Cholesterol'
                ]
data = df_concat2.loc[df_concat2["labItemsNameRef"].isin(_selected_lab)]

In [ ]:
data

In [ ]:
data.loc[(data['labItemsNameRef'] == 'Triglyceride')  ,
         ['labOrderResult','labItemsNormalValueRef','labItemsUnit']].sort_values('labOrderResult')[:50]

In [ ]:
labData = data.loc[:, ["rowID", "cid", "labItemsNameRef", "labOrderResult"]]
labData

In [ ]:
# labData.loc[(labData["labItemsNameRef"] == "Urine-sugar")
#         &(labData["labOrderResult"].isin([""])), 
#         "labOrderResult"] = np.NaN

# labData.loc[(labData["labItemsNameRef"] == "Urine-sugar")
#         & (labData["labOrderResult"].str.contains("n|N|ปกติ|-|Trace|trace")) 
#         & (labData["labItemsNameRef"].notnull()), 
#         "labOrderResult"] = 0

# labData.loc[(labData["labItemsNameRef"] == "Urine-sugar")
#         & (labData["labOrderResult"].str.contains("1")) 
#         & (labData["labItemsNameRef"].notnull()), 
#         "labOrderResult"] = 1

# labData.loc[(labData["labItemsNameRef"] == "Urine-sugar")
#         & (labData["labOrderResult"].str.contains("2")) 
#         & (labData["labItemsNameRef"].notnull()), 
#         "labOrderResult"] = 2

# labData.loc[(labData["labItemsNameRef"] == "Urine-sugar")
#         & (labData["labOrderResult"].str.contains("3")) 
#         & (labData["labItemsNameRef"].notnull()), 
#         "labOrderResult"] = 3

# labData.loc[(labData["labItemsNameRef"] == "Urine-sugar")
#         & (labData["labOrderResult"].str.contains("4")) 
#         & (labData["labItemsNameRef"].notnull()), 
#         "labOrderResult"] = 4


# labData.loc[(labData["labItemsNameRef"] == "Urine-protein-pn")
#         &(labData["labOrderResult"].isin([""])), 
#         "labOrderResult"] = np.NaN

# labData.loc[(labData["labItemsNameRef"] == "Urine-protein-pn")
#         & (labData["labOrderResult"].str.contains("n|N|ปกติ|-|Trace|trace")) 
#         & (labData["labItemsNameRef"].notnull()), 
#         "labOrderResult"] = 0

# labData.loc[(labData["labItemsNameRef"] == "Urine-protein-pn")
#         & (labData["labOrderResult"].str.contains("1")) 
#         & (labData["labItemsNameRef"].notnull()), 
#         "labOrderResult"] = 1

# labData.loc[(labData["labItemsNameRef"] == "Urine-protein-pn")
#         & (labData["labOrderResult"].str.contains("2")) 
#         & (labData["labItemsNameRef"].notnull()), 
#         "labOrderResult"] = 2

# labData.loc[(labData["labItemsNameRef"] == "Urine-protein-pn")
#         & (labData["labOrderResult"].str.contains("3")) 
#         & (labData["labItemsNameRef"].notnull()), 
#         "labOrderResult"] = 3

# labData.loc[(labData["labItemsNameRef"] == "Urine-protein-pn")
#         & (labData["labOrderResult"].str.contains("4")) 
#         & (labData["labItemsNameRef"].notnull()), 
#         "labOrderResult"] = 4

# labData.loc[(labData["labItemsNameRef"] == "Pregnancy Test")
#         &(labData["labOrderResult"].isin(["", "***Pending result***"])), 
#         "labOrderResult"] = np.NaN

# labData.loc[(labData["labItemsNameRef"] == "Pregnancy Test")
#         & (labData["labOrderResult"].str.contains("n|N|ปกติ|-")) 
#         & (labData["labItemsNameRef"].notnull()), 
#         "labOrderResult"] = 0

# # แถวไหนมี Pregnancy Test และ มี P หรือ p นำหน้า และไม่เป็นค่าว่าง จะ assign ให้ result เป็น 1
# labData.loc[(labData["labItemsNameRef"] == "Pregnancy Test")
#         & (labData["labOrderResult"].str.contains("P|p")) 
#         & (labData["labItemsNameRef"].notnull()), 
#         "labOrderResult"] = 1

labData["labOrderResult"] = labData["labOrderResult"].apply(
    lambda x: pd.to_numeric(x, errors= "coerce"))

labData = pl.from_pandas(labData)
# columns= "labName"   values= "labResult" index= "rowID"
labData = labData.pivot(values= "labOrderResult", columns= "labItemsNameRef", index= "rowID", aggregate_fn= "max")
labData = labData.to_pandas()

In [ ]:
labData

In [ ]:
data = df_concat2.iloc[:, 0:-2].drop_duplicates(subset= "rowID") # หลัง pivot

In [ ]:
data = pd.merge(data_col, labData, how= "left", left_on= "rowID", right_on="rowID")

In [ ]:
data

In [ ]:
# เอาเฉพาะ diagType_1 contanis I6 
data = data.explode('diagType_1').reset_index(drop=True)
data = data[(data['diagType_1'] .str.contains("I6"))]
data

In [ ]:
data = data.sort_values(by='visitDateTime', ascending=True)

In [ ]:
data

In [ ]:
# คนไข้มากี่ครั้ง
data['cid'].value_counts().reset_index(name='counts')

In [ ]:
data['diagType_1'].value_counts()

In [ ]:
data

In [ ]:
# a = (data[(data['cid'] == '8d033528c788c66e5286b313ddc9c39c5cebfa873d1689aa9fef0c3ec924a824')][['visitDateTime']].max()[0] - 
#      data[(data['cid'] == '8d033528c788c66e5286b313ddc9c39c5cebfa873d1689aa9fef0c3ec924a824')][['visitDateTime']].min()[0]).days
# a

In [ ]:
# %%time
# # ไล่ตามเเต่ละ row เเล้วคำนวน max-min ของ DateTime ออกมา เพื่อหาจำนวนปี
# data['year'] = [0] * data_c.shape[0]
# for i,r in data.iterrows():  # i:idex ,r:row 
#     data.at[i,'year'] = int((data[(data['cid'] == r['cid'])][['visitDateTime']].max()[0] -
#                              data[(data['cid'] == r['cid'])][['visitDateTime']].min()[0]).days/365) 

In [ ]:
%%time
# สร้าง col ใหม่ขึ้นมาเพื่อเก็บค่า max , min ของ visitDateTime
# agg รวม
df_agg = data.loc[:,['cid','visitDateTime']].groupby(["cid"]).agg(['min','max']).droplevel(0,axis='columns')
df_agg

In [ ]:
# merge กับ data frame
data = pd.merge(data , df_agg, how = 'left' , on=['cid'])

In [ ]:
data

In [ ]:
# data['min_y2'] = data['min_y'].dt.date
# data['max_y2'] = data['max_y'].dt.date

In [ ]:
# def f(x):
#     return x.year
# data['min_y2'] = data['min_y2'].apply(f)
# data['max_y2'] = data['max_y2'].apply(f)

In [ ]:
# def f(x):
#     return x.year
# data['max_y2'] = data['max_y2'].apply(f)

In [ ]:
# ทำให้ col min,max เป็น date เก็บไว้ใน col min_2,max_2
data['min_2'] = data['min'].dt.date
data['max_2'] = data['max'].dt.date

#ทำให้ col min_2 , max_2 เป็น year
def d(x):
    return x.year
data['min_2'] = data['min_2'].apply(d)
data['max_2'] = data['max_2'].apply(d)

In [ ]:
data

In [ ]:
# สร้าง col year 
data['year'] = data['max_2'] - data['min_2']
data

In [ ]:
data[data['cid'] == '4a1d93a977f66de044778ad54463bc516908785fd67ebe6e6299803733bdc623']

In [ ]:
# เช็คว่าส่วนมากผู้ป่วยมาทำอะไร
data[(data['cid'] == 'b2f2bdc360326c5753950eda4644c404b71b6ad3ae1e186550dd4b610c0a0122')][['visits','visitDateTime','vital_cc','year','Cholesterol']].sort_values(by='visits', ascending=True)

In [ ]:
# df2 = data.explode('diagType_2').reset_index(drop=True)
# df2

In [ ]:
# # เลือกเอาเฉพาะคนที่มี visits = 1 because visits = 1 is ผู้ป่วยมาครั้งเเรก
# data1 = data[(data['visits'] == 1)].sort_values(by='visitDateTime', ascending=True)
# data1

In [ ]:
data

## โรคร่วม

In [ ]:
# data1['diagType_1'] = data1['diagType_1'].apply(lambda x: f(x))
# data1['diagType_2'] = data1['diagType_2'].apply(lambda x: f(x))
# data1['diagType_3'] = data1['diagType_3'].apply(lambda x: f(x))

In [ ]:
data

In [ ]:
# .explode() : แตกข้อมูลเเต่ละตัวใน list ออกมาเป็นแถว เพราะในเเต่ละ diagType มีข้อมูลหลายตัวใน list
# value_counts() : นับค่า icd10 ของเเต่ละ diagType
# .reset_index : Index เก่าจะถูกเพิ่มเป็นคอลัมน์ เเละเพิ่ม index ขึ้นมาใหม่
df_count_1 = data['diagType_1'].explode().value_counts().reset_index(name='counts') # name='counts' บอกชื่อ col
df_count_2 = data['diagType_2'].explode().value_counts().reset_index(name='counts')
df_count_3 = data['diagType_3'].explode().value_counts().reset_index(name='counts')

In [ ]:
# top 10 diagType_2

df_count_2.head(10)

# I10 Hypertension / ค.ดันโลหิตสูง
# E785 Hyperlipidaemia / ไขมันในเลือดสูง 
# E119 dm / เบาหวานชนิดที่ 2
# E789 Disorder of lipoprotein metabolism / ความผิดปกติของการเผาผลาญไลโปโปรตีน
# G819 Hemiplegia / อัมพาตครึ่งซีก
# N183 Chronic kidney disease, stage 3 / ไตเรื้อรังระยะ 3
# N182 Chronic kidney disease, stage 2 / ไตเรื้อรังระยะ 2
# E780 Pure hypercholesterolemia / ไขมันในเลือดสูง
# I48  Atrial fibrillation and flutter / ภาวะหัวใจเต้นผิดจังหวะ
# E876 Hypokalemia / ภาวะโพแทสเซียมสูง

In [ ]:
# Create feature columns - for icd10 codes
# เช็คว่ามี str นั้น ๆ อยู่ใน list ไหม ถ้าใช่ก็ 1 ไม่ใช่ก็ 0
def check_substr_list(x_list, substr):
    for i in x_list:
        if substr in i:
            return 1
    return 0
    
def check_multiple_substr(x_list, substr_list):
    for i in x_list:
        for j in substr_list:
            if j in i:
                return 1            
    return 0

In [ ]:
# Extarct ICD10 associated with DM

Hypertension = ['I10', 'I11', 'I12', 'I13', 'I14', 'I16'] # ความดันโลหิตสูง

# hd = ['I21','I210','I211','I212','I213','I214','I219','I22','I220','I221','I228','I229','I23','I231','I232','I233','I234','I235','I236','I238','I24'
#       ,'I240','I241','I248','I249','I25','I250','I252','I253','I254','I255','I259','I258','I256'] # โรคหัวใจ

Hyperlipidaemia = ['E780','E785','E789'] # Hyperlipidaemia / ไขมันในเลือดสูง 

Diabetes_Mellitus = ['E11','E110','E111','E112','E113','E114','E115','E116','E117','E118','E119'] # diabetes type2


Hemiplegia = ['G819'] # Hemiplegia / อัมพาตครึ่งซีก

Chronic_kidney = ['N182','N183'] # Chronic kidney disease / ไตเรื้อรัง

Atrial_fibrillation = ['I48']# Atrial fibrillation and flutter / ภาวะหัวใจเต้นผิดจังหวะ

Hypokalemia = ['E876'] # Hypokalemia / ภาวะโพแทสเซียมสูง


data["icd10_list"] = data["icd10_list"].apply(lambda x: f(x))

data["Hypertension"] = data["icd10_list"].apply(check_multiple_substr, substr_list = Hypertension)
data["Hyperlipidaemia"] = data["icd10_list"].apply(check_multiple_substr, substr_list = Hyperlipidaemia)
data["Diabetes_Mellitus"] = data["icd10_list"].apply(check_multiple_substr, substr_list = Diabetes_Mellitus)
data["Hemiplegia"] = data["icd10_list"].apply(check_multiple_substr, substr_list = Hemiplegia)
data["Chronic_kidney"] = data["icd10_list"].apply(check_multiple_substr, substr_list = Chronic_kidney)
data["Atrial_fibrillation"] = data["icd10_list"].apply(check_multiple_substr, substr_list = Atrial_fibrillation)
data["Hypokalemia"] = data["icd10_list"].apply(check_multiple_substr, substr_list = Hypokalemia)

# data["pcos"] = data["icd10_list"].apply(check_substr_list, substr = 'E282')
# data["hd"] = data["icd10_list"].apply(check_multiple_substr, substr_list = hd)

In [ ]:
# สร้าง col stroke ขึ้นมาโดยมีเงื่อไขว่า ถ้า 'icd10_list' มีตัว I63 อยู่ในนั้น ให้มีค่าเป็น 1 ถ้าไม่มีให้เป็น 0
data['stroke'] = np.where((data['icd10_list'].str.contains("I6")),1,0)
data

In [ ]:
data.drop(columns= ["icd10_list",'vital_cc','vital_pe','diagType_1','diagType_2'], inplace = True)

In [ ]:
data.drop(columns= ['diagType_3'], inplace = True)

In [ ]:
data.to_csv('Clean_1.csv', index= False)

## ทำ feature engineering ดูที่พวก vital_cc 

In [ ]:
# def extract_condition(row, keywords_0, keywords_1):
#     for col in ["vital_cc", "vital_pe", "vital_hpi"]:
#         text = row[col]
#         if text:
#             text = text.lower()
#             match_0 = re.search("|".join(keywords_0), text)
#             match_1 = re.search("|".join(keywords_1), text)
#             if match_0:
#                 return "Yes"
#             elif match_1:
#                 return "No"
#     return None

In [ ]:
# # numbness
# keywords_0 = ["ไม่ชา","ไม่มีชา"]
# keywords_1 = ["ชาแขนซ้าย","มีชา","ชาซีกขวา","ชาตามตัว","ชาปลายมือปลายเท้า","ชาปลายเท้า",
#               "มีอาการชา","เท้าชา","ขาชา","มือ.*ชา","ขาชา","ชาขา","ชาเท้า","ชาที่","รู้สึกชา"
#               ,"ชาแขน","แขนข้างขวาชา","ชาปลายนิ้ว","ชามือ","ชา.*มือ","ชาฝ่าเท้า"]
# IHD["numbness"] = IHD.apply(lambda x: extract_condition(x, keywords_0, keywords_1), axis=1)